In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 22 21:18:07 2018

@author: Adrian
"""

import numpy as np
from datetime import datetime, time
import _sqlite3
import pandas as pd
import json
import re
import boto3
import io

def db_to_csv(query):
    conn = _sqlite3.connect('2016_11.db')
    data = pd.read_sql_query(query, conn)
    data.to_csv('ads.csv',sep=';')
 
def photos_parser(photo_sizes):
    photo_sizes=photo_sizes.replace('""','"')
    data=json.loads(photo_sizes)
    num_of_photos=len(data)
    return(num_of_photos)

def assign_num_of_photos(data):
    all_photos = data['photo_sizes'].apply(str)
    num_of_photos = []
    for i in range(len(all_photos)):
        try:
            num_of_photos.append(photos_parser(all_photos[i]))
        except:
            num_of_photos.append(0)
    return(pd.DataFrame({'num_of_photos': num_of_photos}))


def assign_time_since_creation(data):
    time_since_creation = pd.DataFrame({"time_since_creation": (pd.to_datetime(data['sorting_date'])-pd.to_datetime(data['created_at_first'])).astype('timedelta64[h]')})
    time_since_creation[time_since_creation<0]=0
    return(time_since_creation)

def assign_user_and_city(data):
    city_cnt = Counter()
    user_cnt = Counter() 
    for city in data['city_id']:
        city_cnt[city] += 1
    for user in data['user_id']:
        user_cnt[user] += 1    
    city_ads = data['city_id'].apply(lambda x: city_cnt[x])
    user_ads = data['user_id'].apply(lambda x: user_cnt[x])
    user_and_city = pd.DataFrame({'city_ads': city_ads, 'user_ads': user_ads})
    return(user_and_city)

def assign_traffic_level(data):
    l = np.zeros(len(data))
    twos=np.array((pd.to_datetime(data['sorting_date']).dt.time >= time(18,00)) & (time(22,00) >= pd.to_datetime(data['sorting_date']).dt.time))
    l[twos]=2
    ones=np.array((pd.to_datetime(data['sorting_date']).dt.time >= time(8,00)) & (time(18,00) >= pd.to_datetime(data['sorting_date']).dt.time))
    l[ones]=1
    return(pd.DataFrame({'traffic_level': l}))

def assign_params_parser(params):
    params = str(params)
    par_list=re.split("<=>|<br>", params)
    parsed_params=dict()
    if 'price' in par_list:
        par_list[par_list.index('price')] = "price_type"
    for i in range(0, int((len(par_list)/2))):
        parsed_params[par_list[2*i]]=par_list[2*i+1]
    return parsed_params

def assign_params(data):
    price_type, price, state, Type = ([] for i in range(4))
    for i in range(len(data)):
        params = str(data['params'][i])
        par = params_parser(params)
        try:
            price_type.append(par['price_type'])
        except KeyError:
            price_type.append(0)
        try:
            price.append(par['price'])
        except KeyError:
            price.append(0)
        try:
            state.append(par['state'])
        except KeyError:
            state.append(0)
        try:
            Type.append(par['type'])
        except KeyError:
            Type.append(0)
    params = pd.DataFrame({'price_type': price_type,
                           'price': price,
                           'state': state,
                           'type': Type})
    return(params)
   
def assign_promo_level(data):
    promo_lvl = np.zeros(len(data))
    promo_lvl[data['paidads_id_index']==3] = 1
    promo_lvl[data['paidads_id_index']==4] = 2
    promo_lvl[data['paidads_id_index']==85] = 3
    return(pd.DataFrame({'promotion_level':promo_lvl}))
    
def process_data(df, df_queries):
    params = assign_params(df)
    time_since_creation = assign_time_since_creation(df)
    traffic_level = assign_traffic_level(df)
    num_of_photos = assign_num_of_photos(df)
    promo_level = assign_promo_level(df)
    searches = assign_queries(df, df_queries)
    df = df.drop([ 'id', 'params', 'title','sorting_date', 'created_at_first', 'photo_sizes', 'paidads_valid_to', 'paidads_id_index'],1)
    df = pd.concat([df, params, time_since_creation, traffic_level, num_of_photos, promo_level, searches], axis=1)
    df.loc[df['price_type']=="free",['price']]=0
    return(df)
####################################################################

s3 = boto3.client('s3')
model_raw='model_raw.csv'
sorted_ad_hits = 'sorted_ad_hits.csv'
sorted_empy_sessions = 'sorted_empy_sessions.csv'


obj = s3.get_object(Bucket='2016-11', Key=model_raw)
data_raw = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=';',dtype='unicode')

obj = s3.get_object(Bucket='2016-11', Key=sorted_ad_hits)
sorted_at_hits = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=';',dtype='unicode')

obj = s3.get_object(Bucket='2016-11', Key=sorted_empy_sessions)
sorted_empty_sessions = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=';',dtype='unicode')


In [3]:
data_converted = convert_data(data_raw)
data = pd.concat([data_converted, sorted_at_hits, sorted_empty_sessions],axis=1)


In [6]:
data.to_csv('data.csv',sep=';',index=False)

In [7]:
file = open('data.csv', 'rb')
s3.put_object(Bucket='2016-11', Key='data.csv', Body=file)

{'ETag': '"a445757a99d1563e5b98541d6e84605b"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sun, 25 Feb 2018 17:39:40 GMT',
   'etag': '"a445757a99d1563e5b98541d6e84605b"',
   'server': 'AmazonS3',
   'x-amz-id-2': '2zX6O9hTfZTxyfnbT+xV+i9C0fAnh2J+9FYY57DX1uRACSWln2GpHcDbZSsz3aMTXFYLpGH1yzg=',
   'x-amz-request-id': '05C54B7D46817454'},
  'HTTPStatusCode': 200,
  'HostId': '2zX6O9hTfZTxyfnbT+xV+i9C0fAnh2J+9FYY57DX1uRACSWln2GpHcDbZSsz3aMTXFYLpGH1yzg=',
  'RequestId': '05C54B7D46817454',
  'RetryAttempts': 0}}